# Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import constants
import matplotlib.ticker as ticker
from matplotlib.ticker import EngFormatter
import scipy.io

# Define Essential Element
## Angular Frequency : $\omega=2\pi f $
## Resistor : $Z_{R} = R$
## Constant Phase Element (CPE) :  $Z_{Q} =\frac{1}{Q\times \left ( j\omega  \right )^{\alpha }}$
## Infinite Warburg Element : $Z_{W} =  \frac{\sigma\sqrt{2}}{\sqrt{j\omega }}$

In [ ]:
#############
# Frequency #
#############

#    Define Frequency range in log scale with the out put of Frequency in [Hz] and Angular Frequency in [s^-1]
#    Inputs
#    Initial frequency [Hz]
#    Last frequency    [Hz]
#    Number of point : default is 100 points
#    Outputs (Array)
#    [0] : log scale of Angular Frequency  [s^-1]
#    [1] : log scale of Frequency          [Hz]

def F_range(initial_frequency,last_frequency,number_of_point=100): 
   
    frequency_Hz = np.logspace(np.log10(initial_frequency), np.log10(last_frequency),number_of_point, endpoint=True)
    angular_frequency = 2 * np.pi * frequency_Hz
    
    return angular_frequency, frequency_Hz

############
# Resistor #
############

#   Input
#   resistance : R        [Ohm]    
#   Output
#   Impedance of resistor [Ohm]

def Z_R(resistance):

    resistor_impedance = resistance
    
    return resistor_impedance

################################
# Constant Phase Element (CPE) #
################################

#    Inputs
#    Non ideal capacitance : Q               [s^alpha Ohm^-1]
#    Ideality factor       : alpha           [n/a.]
#    Angular Frequency     : omega           [s^-1]    
#    Output
#    Impedance of the Constant Phase Element [Ohm]

def Z_Q(non_ideal_capacitance,ideality_factor,angular_frequency):

    CPE_impedace = 1/(non_ideal_capacitance * (angular_frequency*1j)**ideality_factor)
    
    return CPE_impedace

################################
# Infinite Warburg Element (W) #
################################

#    Inputs
#    Warburg coefficient : sigma           [Ohm s^-1/2]
#    Angular Frequency   : omega           [s^-1]    
#    Output
#    Impedance of Infinite Warburg Element [Ohm]

def Z_W(sigma,angular_frequency):

    W_impedace =( sigma * np.sqrt(2) ) / np.sqrt(1j*angular_frequency)
    
    return W_impedace


# Define Essential Function

## Random  function

In [ ]:
#######################
# Random in Log space #
#######################

def log_rand(initial_gen,last_gen,size_number):
    
    initial_v = np.log(initial_gen)
    
    last_v = np.log(last_gen)
    
    log_array = np.exp( ( initial_v + ( last_v - initial_v ) * np.random.rand( size_number ) ) )
    
    return log_array

##########################
# Random in Linear space #
##########################

def lin_rand(initial_gen,last_gen,size_number):
  
    lin_array = initial_gen + ( last_gen - initial_gen ) * np.random.rand( size_number )
    
    return lin_array

## Array generator function

In [ ]:
##################################
# Generate resistance (R)) array #
##################################

def genZR(size_number,number_of_point,resistance):

    ZR= np.zeros((size_number,number_of_point), dtype=complex)
    
    for idx in range(size_number):
    
        for idx2 in range(number_of_point): 
    
            ZR[idx][idx2] = Z_R(resistance[idx])
        
    return ZR

###############################################
# Generate Constant Phase Element (CPE) array #
###############################################

def genZQ(size_number,number_of_point,non_ideal_capacitance,ideality_factor,angular_frequency):

    ZQ= np.zeros((size_number,number_of_point), dtype=complex)
    
    for idx in range(size_number):
    
        for idx2 in range(number_of_point): 
    
            ZQ[idx][idx2] = Z_Q(non_ideal_capacitance[idx],ideality_factor[idx],angular_frequency[idx2])
        
    return ZQ

###############################################
# Generate Infinite Warburg Element (W) array #
###############################################

def genZW(size_number,number_of_point,sigma,angular_frequency):

    ZW= np.zeros((size_number,number_of_point), dtype=complex)
    
    for idx in range(size_number):
    
        for idx2 in range(number_of_point): 
    
            ZW[idx][idx2] = Z_W(sigma[idx],angular_frequency[idx2])
        
    return ZW

## Visualization function

In [ ]:
######################
# Set plot formatter #
######################

plt.rcParams['axes.formatter.min_exponent'] = 4
class myformatter(ticker.LogFormatter):

    def _num_to_string(self, x, vmin, vmax):

        if x > 10000:
            s = '%1.0e' % x
        elif x < 1 and x >= 0.001:
            s = '%g' % x
        elif x < 0.001:
            s = '%1.0e' % x
        else:
            s = self._pprint_val(x, vmax - vmin)
        return s
    
xfmt = myformatter(labelOnlyBase=False, minor_thresholds=(4, 0.5))
yfmt = myformatter(labelOnlyBase=False, minor_thresholds=(3, 0.5))

#####################
# Plot EIS spectrum #
#####################

class Zplot:

    def full(ZZ,frequency,param,nrow=1,examp=1):

        px = 1/plt.rcParams['figure.dpi']  # pixel in inches

        fig , axs =plt.subplots(figsize=(700*px, 200*px*nrow),nrows=nrow,ncols=3) 

        if nrow==1:

            phase = np.degrees(np.arctan(ZZ[examp-1].imag/ZZ[examp-1].real))
            mag = np.absolute(ZZ[examp-1])
            
            paramtxt=""
            if param != "" :
              for idx in range(len(param[examp-1])):
                  if idx==4:
                      paramtxt=paramtxt+" \n "                    
                  paramtxt=paramtxt+(format(param[examp-1][idx],".3g"))+" | "           
            
            axs[0].plot(ZZ[examp-1].real,-ZZ[examp-1].imag)
            axs[0].set_title(paramtxt)
            axs[0].set_xlabel("Z' (\u03A9)")
            axs[0].set_ylabel("-Z'' (\u03A9)")

            axs[1].plot(frequency,phase)
            axs[1].set_xscale('log')
            axs[1].set_title("Frequency vs. Phase")
            axs[1].set_xlabel("Frequency (Hz)")
            axs[1].set_ylabel("Phase (\u03B8)")
            axs[1].xaxis.set_minor_formatter(xfmt)

            axs[2].plot(frequency,mag)
            axs[2].set_xscale('log')
            axs[2].xaxis.set_minor_formatter(xfmt)
            axs[2].set_title("Frequency vs. |Z|")
            axs[2].set_xlabel("Frequency (Hz)")
            axs[2].set_ylabel("|Z| (\u03A9)")

            plt.tight_layout()

        elif nrow>1:

            for row in range(nrow):

                fq = frequency
                phase = np.degrees(np.arctan(ZZ[row+examp-1].imag/ZZ[row+examp-1].real))
                mag = np.absolute(ZZ[row+examp-1])

                paramtxt=""
                if param != "" :
                  for idx in range(len(param[row+examp-1])):
                      if idx==4:
                          paramtxt=paramtxt+" \n "   
                      paramtxt=paramtxt+(format(param[row+examp-1][idx],".3g"))+" | "  
                
                
                axs[row,0].text(0.1, 0.9, row+examp, horizontalalignment='center', verticalalignment='center', transform=axs[row,0].transAxes)

                axs[row,0].plot(ZZ[row+examp-1].real,-ZZ[row+examp-1].imag)
                axs[row,0].set_title(paramtxt)
                axs[row,0].set_xlabel("Z' (\u03A9)")
                axs[row,0].set_ylabel("-Z'' (\u03A9)")

                axs[row,1].plot(frequency,phase)
                axs[row,1].set_xscale('log')
                axs[row,1].set_title("Frequency vs. Phase")
                axs[row,1].set_xlabel("Frequency (Hz)")
                axs[row,1].set_ylabel("Phase (\u03B8)")
                axs[row,1].xaxis.set_minor_formatter(xfmt)

                axs[row,2].plot(frequency,mag)
                axs[row,2].set_xscale('log')
                axs[row,2].xaxis.set_minor_formatter(xfmt)
                axs[row,2].set_title("Frequency vs. |Z|")
                axs[row,2].set_xlabel("Frequency (Hz)")
                axs[row,2].set_ylabel("|Z| (\u03A9)")

                plt.tight_layout()
            return

    def point(ZZ,frequency,nrow=1,examp=1):
        
        px = 1/plt.rcParams['figure.dpi']  # pixel in inches

        fig , axs =plt.subplots(figsize=(700*px, 200*px*nrow),nrows=nrow,ncols=4) 

        if nrow==1:

            phase = np.degrees(np.arctan(ZZ[examp-1].imag/ZZ[examp-1].real))
            mag = np.absolute(ZZ[examp-1])                

            axs[0].plot(ZZ[examp-1].real)
            axs[0].set_title("Z'")
            axs[0].set_xlabel("Point")
            axs[0].set_ylabel("Z' (\u03A9)")

            axs[1].plot(-ZZ[examp-1].imag)
            axs[1].set_title("Z''")
            axs[1].set_xlabel("Point")
            axs[1].set_ylabel("-Z'' (\u03A9)")

            axs[2].plot(phase)
            axs[2].set_title("Phase")
            axs[2].set_xlabel("Point")
            axs[2].set_ylabel("Phase (\u03B8)")
            axs[2].xaxis.set_minor_formatter(xfmt)

            axs[3].plot(mag)
            axs[3].xaxis.set_minor_formatter(xfmt)
            axs[3].set_title("|Z|")
            axs[3].set_xlabel("Point")
            axs[3].set_ylabel("|Z| (\u03A9)")

            plt.tight_layout()

        elif nrow>1:

            for row in range(nrow):

                fq = frequency
                phase = np.degrees(np.arctan(ZZ[row+examp-1].imag/ZZ[row+examp-1].real))
                mag = np.absolute(ZZ[row+examp-1])               
                axs[row,0].text(0.1, 0.9, row+examp, horizontalalignment='center', verticalalignment='center', transform=axs[row,0].transAxes)

                axs[row,0].plot(ZZ[row+examp-1].real)
                axs[row,0].set_title("Z'")
                axs[row,0].set_xlabel("Point")
                axs[row,0].set_ylabel("Z' (\u03A9)")

                axs[row,1].plot(-ZZ[row+examp-1].imag)
                axs[row,1].set_title("Z''")
                axs[row,1].set_xlabel("Point")
                axs[row,1].set_ylabel("-Z'' (\u03A9)")

                axs[row,2].plot(phase)
                axs[row,2].set_title("Phase")
                axs[row,2].set_xlabel("Point")
                axs[row,2].set_ylabel("Phase (\u03B8)")
                axs[row,2].xaxis.set_minor_formatter(xfmt)

                axs[row,3].plot(mag)
                axs[row,3].xaxis.set_minor_formatter(xfmt)
                axs[row,3].set_title("|Z|")
                axs[row,3].set_xlabel("Point")
                axs[row,3].set_ylabel("|Z| (\u03A9)")


                plt.tight_layout()

        return


# Circuit simulation

In [ ]:
############
# Circuit1 #
############
def sim_cir1():
    

    R1=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr1= genZR(size_number,number_of_point,R1)


    R2=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr2= genZR(size_number,number_of_point,R2)


    ideality_factor1=np.round(lin_rand(alpha_range[0],alpha_range[1],size_number),3)
    Q1=log_rand(q_range[0],q_range[1],size_number)
    Zq1= genZQ(size_number,number_of_point,Q1,ideality_factor1,angular_frequency)

    Zsum=Zr1 + 1 / ( 1 / Zr2 + 1 / Zq1 ) 
    
    Zparam=[]
    for idx in range(size_number):
        Zparam.append([R1[idx],R2[idx],ideality_factor1[idx],Q1[idx]])
        
    return Zsum,np.array(Zparam)

############
# Circuit2 #
############

def sim_cir2():
    

    R1=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr1= genZR(size_number,number_of_point,R1)


    R2=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr2= genZR(size_number,number_of_point,R2)

    ideality_factor1=np.round(lin_rand(alpha_range[0],alpha_range[1],size_number),3)
    Q1=log_rand(q_range[0],q_range[1],size_number)
    Zq1= genZQ(size_number,number_of_point,Q1,ideality_factor1,angular_frequency)


    R3=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr3= genZR(size_number,number_of_point,R3)


    ideality_factor2=np.round(lin_rand(alpha_range[0],alpha_range[1],size_number),3)
    Q2=log_rand(q_range[0],q_range[1],size_number)
    Zq2= genZQ(size_number,number_of_point,Q2,ideality_factor1,angular_frequency)
    
    Zsum=Zr1 + 1 / ( 1 / Zr2 + 1 / Zq1 ) + 1 / ( 1 / Zr3 + 1 / Zq2 ) 
    
    Zparam=[]
    for idx in range(size_number):
        Zparam.append([R1[idx],R2[idx],R3[idx],ideality_factor1[idx],Q1[idx],ideality_factor2[idx],Q2[idx]])    

    return Zsum,np.array(Zparam)

############
# Circuit3 #
############

def sim_cir3():
    

    R1=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr1= genZR(size_number,number_of_point,R1)

    ideality_factor1=np.round(lin_rand(alpha_range[0],alpha_range[1],size_number),3)
    Q1=log_rand(q_range[0],q_range[1],size_number)
    Zq1= genZQ(size_number,number_of_point,Q1,ideality_factor1,angular_frequency)

    R2=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr2= genZR(size_number,number_of_point,R2)


    sigma=log_rand(sigma_range[0],sigma_range[1],size_number)
    Zw=genZW(size_number,number_of_point,sigma,angular_frequency)
    
    Zsum=Zr1 + 1 / ( 1 / Zq1 + 1 / ( Zr2 + Zw ) )

    Zparam=[]
    for idx in range(size_number):
        Zparam.append([R1[idx],R2[idx],ideality_factor1[idx],Q1[idx],sigma[idx]])    

    return Zsum,np.array(Zparam)

############
# Circuit4 #
############

def sim_cir4():
    

    R1=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr1= genZR(size_number,number_of_point,R1)


    R2=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr2= genZR(size_number,number_of_point,R2)


    ideality_factor1=np.round(lin_rand(alpha_range[0],alpha_range[1],size_number),3)
    Q1=log_rand(q_range[0],q_range[1],size_number)
    Zq1= genZQ(size_number,number_of_point,Q1,ideality_factor1,angular_frequency)


    ideality_factor2=np.round(lin_rand(alpha_range[0],alpha_range[1],size_number),3)
    Q2=log_rand(q_range[0],q_range[1],size_number)
    Zq2= genZQ(size_number,number_of_point,Q2,ideality_factor1,angular_frequency)


    R3=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr3= genZR(size_number,number_of_point,R3)

    sigma=log_rand(sigma_range[0],sigma_range[1],size_number)
    Zw=genZW(size_number,number_of_point,sigma,angular_frequency)
    
    Zsum=Zr1 + 1 / ( 1 / Zr2 + 1 / Zq1 ) + 1 / ( 1 / Zq2 + 1 / ( Zr3 + Zw ) )
    
    Zparam=[]
    for idx in range(size_number):
        Zparam.append([R1[idx],R2[idx],R3[idx],ideality_factor1[idx],Q1[idx],ideality_factor2[idx],Q2[idx],sigma[idx]])    

    return Zsum,np.array(Zparam)

############
# Circuit5 #
############

def sim_cir5():
    

    R1=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr1= genZR(size_number,number_of_point,R1)


    R2=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr2= genZR(size_number,number_of_point,R2)

    ideality_factor1=np.round(lin_rand(alpha_range[0],alpha_range[1],size_number),3)
    Q1=log_rand(q_range[0],q_range[1],size_number)
    Zq1= genZQ(size_number,number_of_point,Q1,ideality_factor1,angular_frequency)

    R3=log_rand(resistance_range[0],resistance_range[1],size_number)
    Zr3= genZR(size_number,number_of_point,R3)


    ideality_factor2=np.round(lin_rand(alpha_range[0],alpha_range[1],size_number),3)
    Q2=log_rand(q_range[0],q_range[1],size_number)
    Zq2= genZQ(size_number,number_of_point,Q2,ideality_factor1,angular_frequency)


    sigma=log_rand(sigma_range[0],sigma_range[1],size_number)
    Zw=genZW(size_number,number_of_point,sigma,angular_frequency)
    
    Zsum=Zr1 + 1 / ( 1 / ( Zr2 +  1 / (( 1 / (Zr3 + Zw )) + 1 / Zq2 ) ) + 1 / Zq1 ) 
    
    Zparam=[]
    for idx in range(size_number):
        Zparam.append([R1[idx],R2[idx],R3[idx],ideality_factor1[idx],Q1[idx],ideality_factor2[idx],Q2[idx],sigma[idx]])    

    return Zsum,np.array(Zparam)


# EIS data simulation

In [ ]:
########################
# Initialize parameter #
########################

#Number of circuit
number_of_circuit=5    

#Number of spectrum in each circuit : 256 512 1024 2048 4096 8192 16384 32768
size_number=4096

#Numer of data point in each spectrum
number_of_point=100   

#Range of frequency
angular_frequency=F_range(0.01,1000000,number_of_point)[0] 

#Range of resistance
resistance_range=[10**-1,10**4] 

#Range of idality factor of CPE
alpha_range=[0.8,1] 

#Range of CPE capacitance
q_range=[10**-5,10**-3]

#Range of sigma
sigma_range=[10**0,10**3]

Circuit_spec=np.zeros((number_of_circuit,size_number,number_of_point), dtype=complex)


Circuit_spec[0],Circuit0_param=sim_cir1()
Circuit_spec[1],Circuit1_param=sim_cir2()
Circuit_spec[2],Circuit2_param=sim_cir3()
Circuit_spec[3],Circuit3_param=sim_cir4()
Circuit_spec[4],Circuit4_param=sim_cir5()

param_dict={"Circuit0_param":Circuit0_param,"Circuit1_param":Circuit1_param,"Circuit2_param":Circuit2_param,"Circuit3_param":Circuit3_param,"Circuit4_param":Circuit4_param}



# DATA EXPORT
## Data export function

In [ ]:
def arrange_data(Circuit,cir_class,size_number,number_of_point):
    
    label = cir_class
    imge = Circuit.imag
    phase = np.degrees(np.arctan(Circuit.imag/Circuit.real))
    mag = np.absolute(Circuit)
    
    x= np.zeros((size_number,3,number_of_point))
    y= np.zeros(size_number)
    
    for idx in range(size_number):
            
            y[idx] = cir_class
            
            for idx2 in range(number_of_point): 
                
                x[idx][0][idx2] = imge[idx][idx2]
                x[idx][1][idx2] = phase[idx][idx2]
                x[idx][2][idx2] = mag[idx][idx2]
                
   
    return x,y

def export_data(Circuit,size_number,number_of_point,numc):    
    
    x= np.zeros((numc,size_number,3,number_of_point))
    y= np.zeros((numc,size_number))
    
    for idx in range(numc):
        x[idx],y[idx]=arrange_data(Circuit[idx],(idx),size_number,number_of_point)
    
    x_data=x[0]
    y_data=y[0]
    
    for idx in range(numc-1):
        x_data=np.append(x_data,x[idx+1],axis=0)
        
    for idx in range(numc-1):
        y_data=np.append(y_data,y[idx+1],axis=0)
    
    return x_data,y_data

## Export data

In [ ]:
all_param=[]
all_param.append(Circuit0_param.tolist())
all_param.append(Circuit1_param.tolist())
all_param.append(Circuit2_param.tolist())
all_param.append(Circuit3_param.tolist())
all_param.append(Circuit4_param.tolist())
df = pd.DataFrame(all_param)
df.transpose()
k=df.stack()
k.to_csv('paramc1.csv', index=False)

In [ ]:
dat_ver="v2"
if size_number >= 1000: data_num_n = str("%.0f%s" % (size_number/1000.0, 'k'))
else : data_num_n = str(size_number)

File_name="xy_data_"+data_num_n+"_"+str(number_of_circuit)+"circuit_"+dat_ver+".mat"

x_data,y_data = export_data(Circuit_spec,size_number,number_of_point,numc=number_of_circuit)
mdic={"x_data":x_data,"y_data":y_data}
scipy.io.savemat(File_name, mdic)

In [ ]:
# Dulyawat Doonyapisut Email:charting9@gmail.com